<a href="https://colab.research.google.com/github/pedrambayat/MIC_prediction/blob/main/alphafold_csv_generation_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
!sudo apt-get update
!sudo apt-get install -y dssp

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,201 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,842 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-se

In [ ]:
import zipfile
import json
from pathlib import Path
import numpy as np
import pandas as pd

from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
from Bio.PDB.Polypeptide import protein_letters_3to1 as three_to_one_dict
from Bio.PDB.vectors import calc_dihedral, Vector
from scipy.spatial.distance import cdist
import re

In [ ]:
# -------------------------------
# INPUT AND OUTPUT SETTINGS
# -------------------------------
ZIP_FOLDER = Path("/content/drive/MyDrive/CIS520FinalProjectData/Extinct_Data")        # folder containing AlphaFold *.zip files
EXTRACT_TMP = Path("/content/drive/MyDrive/CIS520FinalProjectData/Extinct_Data/temp")               # temporary extraction folder
OUTPUT_FOLDER = Path("/content/drive/MyDrive/CIS520FinalProjectData/extinct_csvs")            # final CSVs go here

EXTRACT_TMP.mkdir(exist_ok=True)
OUTPUT_FOLDER.mkdir(exist_ok=True)

In [ ]:
# -------------------------------
# MAIN PROCESSING LOOP
# -------------------------------
import shutil

for zip_path in ZIP_FOLDER.glob("*.zip"):

    print(f"Processing {zip_path.name}...")

    # Clear the extract folder every time
    if EXTRACT_TMP.exists():
        shutil.rmtree(EXTRACT_TMP)
    EXTRACT_TMP.mkdir(parents=True, exist_ok=True)

    # Extract fresh
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(EXTRACT_TMP)

    pdb_files = list(EXTRACT_TMP.glob("*.pdb"))
    print("Found PDBs:", [p.name for p in pdb_files])

    ranked_models = []

    # Match _rank_###_ in filename
    for pdb in pdb_files:
        m = re.search(r"_rank_(\d+)_", pdb.name)
        if m:
            ranked_models.append((int(m.group(1)), pdb))

    if not pdb_files:
        print("  ERROR: No PDB files found in zip. Skipping.")
        continue

    if not ranked_models:
        print("  WARNING: No ranked PDBs found. Using first.")
        pdb_file = pdb_files[0]
    else:
        ranked_models.sort(key=lambda x: x[0])  # lower rank = better
        best_rank, pdb_file = ranked_models[0]

    print("Selected model:", pdb_file.name)

    protein_name = zip_path.stem
    pdb_path = pdb_file

    # -------------------------------
    # Parse PDB
    # -------------------------------
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(protein_name, pdb_path)

    model = structure[0]
    chain = list(model.get_chains())[0]
    residues = list(chain.get_residues())
    print(len(residues))
    residues = [res for res in chain if res.has_id("CA")]
    print(residues)

    sequence = ""
    coords = []
    plddt = []

    for res in residues:
        try:

            aa = three_to_one_dict[res.resname]
        except Exception:
            print("Invalid AA found")
            aa = "X"
        sequence += aa
        coords.append(res["CA"].coord)
        plddt.append(res["CA"].bfactor)

    coords = np.array(coords)
    L = len(residues)

    # -------------------------------
    # DSSP for secondary structure + SASA
    # -------------------------------
    try:
        dssp = DSSP(model, pdb_path)
        sec_struct = []
        sasa = []

        # DSSP iterates in order of residues
        for key in dssp.keys():
            sec_struct.append(dssp[key][2])   # H/E/C/…
            sasa.append(dssp[key][3])
        print("  DSSP succeeded, there is secondary structure.")
    except Exception:
        sec_struct = [None] * L
        sasa = [None] * L

    # -------------------------------
    # Compute torsions (phi/psi)
    # -------------------------------
    phi = []
    psi = []

    for i in range(L):
        try:
            # phi(i) = C(i-1), N(i), CA(i), C(i)
            if i > 0:
                c_prev = residues[i-1]["C"].coord
                n = residues[i]["N"].coord
                ca = residues[i]["CA"].coord
                c = residues[i]["C"].coord
                phi_val = calc_dihedral(Vector(c_prev), Vector(n), Vector(ca), Vector(c)) * 180 / np.pi
            else:
                phi_val = None

            # psi(i) = N(i), CA(i), C(i), N(i+1)
            if i < L - 1:
                n = residues[i]["N"].coord
                ca = residues[i]["CA"].coord
                c = residues[i]["C"].coord
                n_next = residues[i+1]["N"].coord
                psi_val = calc_dihedral(Vector(n), Vector(ca), Vector(c), Vector(n_next)) * 180 / np.pi
            else:
                psi_val = None

            phi.append(phi_val)
            psi.append(psi_val)

        except Exception:
            phi.append(None)
            psi.append(None)

    # -------------------------------
    # Contact map (Cα distances < 8Å)
    # -------------------------------

    try:
        dist_matrix = cdist(coords, coords)
        contact_map = (dist_matrix < 8.0).astype(int)
        contact_dict = {i: contact_map[i].tolist() for i in range(L)}  # dictionary form
    except Exception:
        print("  Contact map failed.")
        contact_dict = {i: None for i in range(L)}

    # -------------------------------
    # Build DataFrame
    # -------------------------------
    df = pd.DataFrame({
        "residue_index": np.arange(1, L+1),
        "aa": list(sequence),
        "pLDDT": plddt,
        "sec_struct": sec_struct,
        "solvent_acc": sasa,
        "phi": phi,
        "psi": psi,
    })

    # Add dictionary column (same dict for each row)
    df["contact_map"] = [contact_dict] * L

    # Metadata (optional)
    df["protein_name"] = protein_name
    df["sequence"] = sequence
    print(df)
    # -------------------------------
    # Save CSV
    # -------------------------------
    out_path = OUTPUT_FOLDER / f"{protein_name}.csv"
    df.to_csv(out_path, index=False)

    print(f"  Saved → {out_path}")

    # Clean temporary directory
    for f in EXTRACT_TMP.iterdir():
        if f.is_file():
            f.unlink()
        else:
            # Recursively delete folders
            import shutil
            shutil.rmtree(f)

Processing UEP15361.result.zip...
Found PDBs: ['UEP15361.1-KKW10_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb', 'UEP15361.1-KKW10_unrelaxed_rank_002_alphafold2_ptm_model_1_seed_000.pdb', 'UEP15361.1-KKW10_unrelaxed_rank_003_alphafold2_ptm_model_3_seed_000.pdb', 'UEP15361.1-KKW10_unrelaxed_rank_004_alphafold2_ptm_model_5_seed_000.pdb', 'UEP15361.1-KKW10_unrelaxed_rank_005_alphafold2_ptm_model_2_seed_000.pdb']
Selected model: UEP15361.1-KKW10_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb
10
[<Residue LYS het=  resseq=1 icode= >, <Residue LYS het=  resseq=2 icode= >, <Residue TRP het=  resseq=3 icode= >, <Residue THR het=  resseq=4 icode= >, <Residue LYS het=  resseq=5 icode= >, <Residue ILE het=  resseq=6 icode= >, <Residue TYR het=  resseq=7 icode= >, <Residue SER het=  resseq=8 icode= >, <Residue PRO het=  resseq=9 icode= >, <Residue LEU het=  resseq=10 icode= >]


/usr/local/lib/python3.12/dist-packages/Bio/PDB/DSSP.py:199: UserWarning: Error parsing PDB at line 1
Error trying to load file "/content/drive/MyDrive/CIS520FinalProjectData/Extinct_Data/temp/UEP15361.1-KKW10_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb"
Expected record CRYST1 but found MODEL 

  warnings.warn(err)


   residue_index aa  pLDDT sec_struct solvent_acc        phi         psi  \
0              1  K  68.62       None        None        NaN   81.304384   
1              2  K  77.44       None        None -60.689099   95.048184   
2              3  W  79.69       None        None -59.408283   93.782824   
3              4  T  80.69       None        None -79.489964  108.195173   
4              5  K  81.38       None        None -59.725852  110.938034   
5              6  I  82.00       None        None -75.160836  105.673554   
6              7  Y  80.88       None        None -77.908255   96.989253   
7              8  S  78.62       None        None -69.727856   93.171245   
8              9  P  72.62       None        None -67.330597   39.461955   
9             10  L  65.38       None        None -92.802027         NaN   

                                         contact_map     protein_name  \
0  {0: [1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 1: [1, 1, ...  UEP15361.result   
1  {0: [1, 1, 1, 

/usr/local/lib/python3.12/dist-packages/Bio/PDB/DSSP.py:199: UserWarning: Error parsing PDB at line 1
Error trying to load file "/content/drive/MyDrive/CIS520FinalProjectData/Extinct_Data/temp/BAF96956.1-KTL10_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb"
Expected record CRYST1 but found MODEL 

  warnings.warn(err)


   residue_index aa  pLDDT sec_struct solvent_acc         phi        psi  \
0              1  K  72.50       None        None         NaN  -1.869616   
1              2  T  78.75       None        None  -91.309209 -15.835227   
2              3  L  82.25       None        None  -77.623652 -25.670548   
3              4  K  86.38       None        None  -76.337615 -25.085811   
4              5  I  88.25       None        None  -79.094406 -23.441550   
5              6  I  90.38       None        None  -84.265667 -17.594394   
6              7  R  88.19       None        None  -76.002567 -17.020731   
7              8  L  80.88       None        None  -93.351544 -20.042330   
8              9  L  79.38       None        None  -96.951408   0.807489   
9             10  F  63.16       None        None -103.698613        NaN   

                                         contact_map     protein_name  \
0  {0: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], 1: [1, 1, ...  BAF96956.result   
1  {0: [1, 1, 1, 

...

/usr/local/lib/python3.12/dist-packages/Bio/PDB/DSSP.py:199: UserWarning: Error parsing PDB at line 1
Error trying to load file "/content/drive/MyDrive/CIS520FinalProjectData/Extinct_Data/temp/ABG37012.1-RAC26_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb"
Expected record CRYST1 but found MODEL 

  warnings.warn(err)


    residue_index aa  pLDDT sec_struct solvent_acc         phi        psi  \
0               1  R  77.62       None        None         NaN -25.806238   
1               2  A  84.94       None        None  -69.042154 -34.653604   
2               3  C  85.88       None        None  -70.839615 -33.073946   
3               4  L  88.00       None        None  -70.238515 -35.020899   
4               5  H  88.62       None        None  -62.205899 -42.202912   
5               6  A  90.50       None        None  -65.891566 -37.174215   
6               7  R  89.44       None        None  -68.954253 -35.852422   
7               8  S  90.06       None        None  -65.559826 -37.996161   
8               9  I  89.62       None        None  -69.449326 -40.846045   
9              10  A  87.44       None        None  -62.137625 -41.728111   
10             11  R  88.81       None        None  -65.998400 -38.412582   
11             12  L  87.12       None        None  -62.138659 -44.426301   

/usr/local/lib/python3.12/dist-packages/Bio/PDB/DSSP.py:199: UserWarning: Error parsing PDB at line 1
Error trying to load file "/content/drive/MyDrive/CIS520FinalProjectData/Extinct_Data/temp/ADN88909.1-RAY26_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb"
Expected record CRYST1 but found MODEL 

  warnings.warn(err)


    residue_index aa  pLDDT sec_struct solvent_acc        phi        psi  \
0               1  R  74.31       None        None        NaN -29.288149   
1               2  A  82.81       None        None -67.631855 -31.845775   
2               3  Y  85.62       None        None -73.891996 -40.671218   
3               4  I  83.94       None        None -65.038758 -37.239025   
4               5  C  85.44       None        None -67.374026 -38.956386   
5               6  R  91.06       None        None -65.174926 -40.876927   
6               7  K  92.75       None        None -66.278843 -39.148515   
7               8  K  93.31       None        None -67.154742 -40.213773   
8               9  F  93.31       None        None -65.010678 -42.654690   
9              10  L  94.31       None        None -61.174786 -39.671336   
10             11  S  94.62       None        None -64.862310 -43.316721   
11             12  L  93.56       None        None -63.108295 -40.910297   
12          

/usr/local/lib/python3.12/dist-packages/Bio/PDB/DSSP.py:199: UserWarning: Error parsing PDB at line 1
Error trying to load file "/content/drive/MyDrive/CIS520FinalProjectData/Extinct_Data/temp/AWH62785.1-RLA27_unrelaxed_rank_001_alphafold2_ptm_model_3_seed_000.pdb"
Expected record CRYST1 but found MODEL 

  warnings.warn(err)
